In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
scaler = TfidfVectorizer()

In [3]:
train_t = pd.read_csv('train.csv')
val_t = pd.read_csv('val.csv')
test_t = pd.read_csv('test.csv')

In [4]:
Train = train_t#.append(val_t)
Train

,product_id,reference_id,merged
0,0007302f2fe1d54d,f497219eb0077f84,Классическая сплит-система ROYAL CLIMA PANDORA...
1,0039af5efceac4ab,28085e941cde1639,Холодильник Бирюса 118. Мощность замораживани...
2,004f2158acb8165c,9afe55bb4bf1e8a8,ASUS TUF-GTX1660S-O6G-GAMING Видеокарта. Объем...
3,005cddb29e1677ec,1f21918ceb5d345c,"Кофемашина Saeco Lirika One Touch Cappuccino, ..."
4,0083dddef6bc6503,8f9c8fcf9cb2862c,Lenovo 62A8KAT1EU Монитор. Яркость 250 кд/м²...
...,...,...,...
1812,fa06227ed7b42bba,79365b00febf391b,Посудомоечная машина Weissgauff BDW 4004. Тайм...
1813,fa3b915603e2839e,13a23947cc15cfe7,Монитор Xiaomi Mi Desktop Monitor 27'' RMMNT27...
1814,fc7ac235b0c1149c,b435b4515808e799,Холодильник GA-B379SLUL LG. Общий объем 261 л....
1815,feb6f146f3be1ccb,8e2012bfdbccf188,Водонагреватель ariston накопительный водонагр...


In [5]:
len(train_t), len(val_t), len(train_t) + len(val_t)

(1817, 409, 2226)

In [6]:
Val = val_t

In [7]:
Test = test_t

In [8]:
X_train = Train['merged'].to_numpy()
X_val = Val['merged'].to_numpy()
X_test = Test['merged'].to_numpy()

y_train = Train['reference_id'].to_numpy()
y_val = Val['reference_id'].to_numpy()
y_test = Test['reference_id'].to_numpy()

In [9]:
encoder = {i: k for k, i in enumerate(np.unique(y_train))}

In [10]:
scaler.fit(X_train)

TfidfVectorizer()

# Neural Net

In [11]:
import numpy as np
from keras.datasets import mnist
from torch.utils.data import Dataset
from typing import Optional
import os
import numpy as np
import torch
from torchvision import transforms, utils
from torchvision.utils import make_grid
from torch import nn
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from matplotlib import pyplot as plt
import torchvision.transforms as T

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


## Версия без испльзования описания эталонов

In [12]:
class MyCustomDataset(Dataset):
    def __init__(self,
                 X,
                 y,
                 scaler = scaler,
                 encoder = encoder
                ):
        ## list of tuples: (img, label)
        self._items = []
        self.scaler = scaler
        self.encoder = encoder
        for (text, label) in zip(self.scaler.transform(X).toarray(), y):
            self._items.append((text, self.encoder[label]))        

    def __len__(self):
        return len(self._items)

    def __getitem__(self, index):
        text, label = self._items[index]

        return text, label

In [13]:
ds_train = MyCustomDataset(X_train, y_train)
ds_val = MyCustomDataset(X_val, y_val)
ds_test = MyCustomDataset(X_test, y_test)

In [14]:
ds_train[0][0].shape[0]

3657

In [15]:
len(ds_train), len(ds_val), len(ds_test)

(1817, 409, 554)

In [16]:
dl_train = DataLoader(ds_train, batch_size=32, shuffle=True, num_workers=0)
dl_val = DataLoader(ds_val, batch_size=32, shuffle=False, num_workers=0)
dl_test = DataLoader(ds_test, batch_size=32, shuffle=False, num_workers=0)

In [68]:
class MyModel(pl.LightningModule):
    # REQUIRED
    def __init__(self, input_shape=ds_train[0][0].shape[0], test_data=dl_test, num_classes=len(encoder.keys())):
        super().__init__()
        """ Define computations here. """
        
        # Описываем модель
        self.fc1 = nn.Linear(input_shape, 2048)
        self.act1 = nn.LeakyReLU(negative_slope=0.1)
        self.drop1 = nn.Dropout(p=0.7)
        self.fc2 = nn.Linear(2048, 512)
        self.act2 = nn.LeakyReLU(negative_slope=0.05)
        self.drop2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(512, num_classes)
        
        # Для оценки качества на тестовых данных можем указать 
        # сразу данные, на которых и будет производиться эта оценка
        self.test_data = test_data
        
        # В качестве ошибки выступает кросс-энтропия
        self.loss = F.cross_entropy
        
        # Дополнительно посмотрим на метрику точности, т.к. она более понятна
        self.accuracy = lambda x, y: (x.argmax(-1) == y).float().mean()
    
    # REQUIRED
    def forward(self, x):
        """ Use for inference only (separate from training_step). """
        # Чтобы избежать ошибок, переводим данные в нужный формат
        x = x.type(torch.FloatTensor)
        if torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
        
        # Описываем поведение сети, т.е. как она проходит 
        # через 1-ый слой, 
        x = self.drop1(self.act1(self.fc1(x)))
        # применяем функцию-срезку
        x = self.drop2(self.act2(self.fc2(x)))
        
        #x = self.drop3(self.act3(self.fc3(x)))
        # применяем второй слой
        x = self.fc3(x)
    
        return x
    
    # REQUIRED -- в конце каждого этапа обучения будут сохраняться результаты 
    # ошибок и точности на данной эпохе для обуающей выборки
    def training_step(self, batch, batch_idx):
        """the full training loop"""
        x, y = batch
        
        x = x.type(torch.FloatTensor)
        y = y.type(torch.LongTensor)
        if torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.LongTensor)

        y_logit = self(x)
        loss = self.loss(y_logit, y)
        
        acc = self.accuracy(y_logit, y)

        return {'loss': loss, 'acc': acc}

    # REQUIRED -- определяем оптимизатор и задаем learning rate
    # P.s. еще можно указать как менять скорость обучения, 
    #      но тут мало эпох и слишком простая задача для этого
    def configure_optimizers(self):
        """ Define optimizers and LR schedulers. """
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                                  mode='min', 
                                                                  factor=0.2, 
                                                                  patience=2, 
                                                                  verbose=True)
        
        lr_dict = {
            # REQUIRED: The scheduler instance
            "scheduler": lr_scheduler,
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "epoch",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
            # Metric to to monitor for schedulers like `ReduceLROnPlateau`
            "monitor": "val_loss"
        } 
        
        return [optimizer], [lr_dict]
    
    # OPTIONAL -- как и с обучающими, но тут результаты на валидации, чтобы отслеживать переобучение
    def validation_step(self, batch, batch_idx):
        """the full validation loop"""
        x, y = batch

        x = x.type(torch.FloatTensor)
        y = y.type(torch.LongTensor)
        if torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.LongTensor)
            
        y_logit = self(x)
        loss = self.loss(y_logit, y)
        
        acc = self.accuracy(y_logit, y)

        return {'val_loss': loss, 'val_acc': acc}

    # OPTIONAL -- как раз визуализация среднего значения результатов на обучающей выборке
    def training_epoch_end(self, outputs):
        """log and display average train loss and accuracy across epoch"""
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
        
        print(f"| Train_acc: {avg_acc:.2f}, Train_loss: {avg_loss:.2f}" )
        
        self.log('train_loss', avg_loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('train_acc', avg_acc, prog_bar=True, on_epoch=True, on_step=False)
     
    # OPTIONAL -- то же самое для валидации
    def validation_epoch_end(self, outputs):
        """log and display average val loss and accuracy"""
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        
        print(f"[Epoch {self.trainer.current_epoch:3}] Val_loss: {avg_loss:.5f} Val_accuracy: {avg_acc:.5f}", end= " ")
        
        self.log('val_loss', avg_loss, prog_bar=True, on_epoch=True, on_step=False)
        self.log('val_acc', avg_acc, prog_bar=True, on_epoch=True, on_step=False)
    
    # Оценка качества на тестовой выборке
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        acc = self.accuracy(logits, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        
    # Какие данные сеть полагает тестовыми
    def test_dataloader(self):
        return self.test_data

In [69]:
m = MyModel()

In [70]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [71]:
trainer.fit(m, dl_train, dl_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | fc1   | Linear    | 7.5 M 
1 | act1  | LeakyReLU | 0     
2 | drop1 | Dropout   | 0     
3 | fc2   | Linear    | 1.0 M 
4 | act2  | LeakyReLU | 0     
5 | drop2 | Dropout   | 0     
6 | fc3   | Linear    | 241 K 
------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.129    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[Epoch   0] Val_loss: 6.15321 Val_accuracy: 0.01562 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

[Epoch  12] Val_loss: 0.31165 Val_accuracy: 0.91865 | Train_acc: 0.94, Train_loss: 0.23


Validation: 0it [00:00, ?it/s]

[Epoch  13] Val_loss: 0.25673 Val_accuracy: 0.92587 | Train_acc: 0.97, Train_loss: 0.17


Validation: 0it [00:00, ?it/s]

[Epoch  14] Val_loss: 0.24211 Val_accuracy: 0.93788 | Train_acc: 0.96, Train_loss: 0.15


In [72]:
trainer.test()

Restoring states from the checkpoint path at /mnt/d/AgoraHack/lightning_logs/version_33/checkpoints/epoch=14-step=855.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /mnt/d/AgoraHack/lightning_logs/version_33/checkpoints/epoch=14-step=855.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9187725782394409
        test_loss            0.294485867023468
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.294485867023468, 'test_acc': 0.9187725782394409}]

In [17]:
from sklearn import neighbors
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

In [18]:
from sklearn.metrics import accuracy_score

# SiamsNet

In [17]:
class ArcLoss(nn.Module):
    def __init__(self, in_features, out_features):
        super(ArcLoss, self).__init__()
        self.s = 30.0
        self.m = 0.4
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.eps = 1e-7

    def forward(self, x, labels):
        for W in self.fc.parameters():
            W = F.normalize(W, p=2, dim=1)
        x = F.normalize(x, p=2, dim=1)
        cos_theta = self.fc(x)
        
        numerator = cos_theta.transpose(0, 1)
        numerator = torch.diagonal(numerator[labels])
        numerator = torch.clamp(numerator, -1+self.eps, 1-self.eps)
        numerator = self.s*torch.cos(torch.acos(numerator)+self.m)
        
        excluded_real_class = torch.cat([
            torch.cat([cos_theta[i, :y], cos_theta[i, y+1:]])[None, :]
            for i, y in enumerate(labels)
        ], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s*excluded_real_class), dim=1)
        
        loss = numerator - torch.log(denominator)
        return cos_theta, -loss.mean()

In [18]:
class ArcNet(pl.LightningModule):
    def __init__(self, emb_net, train_classes):
        super().__init__()
        self.emb_net = emb_net
        self.classifier = ArcLoss(512, train_classes)

    def forward(self, x, cudaaa = True):
        x = x.type(torch.FloatTensor)
        if cudaaa and torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
        
        output = self.emb_net(x)
        output = self.classifier(output)
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        
        x = x.type(torch.FloatTensor)
        y = y.type(torch.LongTensor)
        if torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.LongTensor)
            
        outputs = self.emb_net(x)
        outputs, loss = self.classifier(outputs, y)
        acc = (outputs.argmax(dim=1) == y).sum().item() / len(outputs)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [19]:
class EmbeddingNet(nn.Module):
    def __init__(self, input_shape=ds_train[0][0].shape[0]):
        super(EmbeddingNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_shape, 2048),
            nn.LeakyReLU(negative_slope=0.3),
            nn.Dropout(p=0.6),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(negative_slope=0.15),
            nn.Dropout(p=0.4),
            nn.Linear(1024, 512)
        )

    def forward(self, x, cudaaa = True):
        x = x.type(torch.FloatTensor)
        if cudaaa and torch.cuda.is_available():
            x = x.type(torch.cuda.FloatTensor)
            
        output = self.net(x)
        return output

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [21]:
def train_knn(emb_net, is_normalize=False):
    data_x = []
    data_y = []
    with torch.no_grad():
        for data in dl_train:
            inputs, labels = data[0], data[1]
            feats = emb_net(inputs, cudaaa=False).detach().cpu().numpy()
            
            data_x.append(feats)
            data_y.append(labels)
    data_x = np.concatenate(data_x, axis=0)
    data_y = np.concatenate(data_y, axis=0)
    
    knn = KNeighborsClassifier(n_neighbors=1, metric='cosine') #512 -> 1 эталон -- предложение -> NeuralNet[3657 -> 512] Knn[512 -> 1] эталон
    knn = knn.fit(data_x, data_y)
    
    return knn

In [22]:
def test_knn(emb_net, knn, is_normalize=False):
    total_correct = 0
    total_cnt = 0
    with torch.no_grad():
        for data in dl_test:
            inputs, labels = data[0], data[1].detach().cpu().numpy()
            feats = emb_net(inputs, cudaaa=False).detach().cpu().numpy()
            preds = knn.predict(feats)
            
            total_correct += (preds == labels).sum()
            total_cnt += len(preds)
    
    acc = total_correct/total_cnt
    print(f'Accuracy = {acc*100}%')
    return acc

In [ ]:
arc_train_dl = DataLoader(ds_train, 32, num_workers=0)
arc_val_dl = DataLoader(dl_val, 32, num_workers=0)

In [37]:
net_arc = ArcNet(EmbeddingNet(), train_classes=len(encoder.keys()))
trainer = pl.Trainer(gpus=1, max_epochs=5)
trainer.fit(net_arc, arc_train_dl, arc_val_dl)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/rw/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:131: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | emb_net    | EmbeddingNet | 10.1 M
1 | classifier | ArcLoss      | 241 K 
--------------------------------------------
10.4 M    Trainable params
0         Non-trainable params
10.4 M    Total params
41.423    Total estimated model params size (MB)
/home/rw/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have m

Training: 0it [00:00, ?it/s]

In [49]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) # RF

Accuracy = 82.12996389891697%


0.8212996389891697

In [51]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #KNN 1

Accuracy = 94.22382671480143%


0.9422382671480144

In [53]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #KNN 3

Accuracy = 93.14079422382672%


0.9314079422382672

In [55]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #KNN 5

Accuracy = 93.14079422382672%


0.9314079422382672

In [23]:
L_train = pd.concat([train_t, val_t])
L_train

,product_id,reference_id,merged
0,0007302f2fe1d54d,f497219eb0077f84,Классическая сплит-система ROYAL CLIMA PANDORA...
1,0039af5efceac4ab,28085e941cde1639,Холодильник Бирюса 118. Мощность замораживани...
2,004f2158acb8165c,9afe55bb4bf1e8a8,ASUS TUF-GTX1660S-O6G-GAMING Видеокарта. Объем...
3,005cddb29e1677ec,1f21918ceb5d345c,"Кофемашина Saeco Lirika One Touch Cappuccino, ..."
4,0083dddef6bc6503,8f9c8fcf9cb2862c,Lenovo 62A8KAT1EU Монитор. Яркость 250 кд/м²...
...,...,...,...
404,f64cac93254ce4d5,f50c3e7f0b02facb,Видеокарта MSI GeForce RTX 2060 1680Mhz PCI-E ...
405,f6845400f1cf53d8,25d66337f5007356,Смартфон Samsung Galaxy M52 5G 8/128 ГБ Global...
406,f8379ac458f614c1,3506a28b65ac9e2b,Водонагреватель thermex titaniumheat 150v 1110...
407,f975127890a04b50,35b3667e9c4d5ab3,"Водонагреватель thermex surf 5000, 5 квт, с ле..."


In [24]:
L_train = np.stack([*X_train, *X_val], axis=0)
Ly_train = np.stack([*y_train, *y_val], axis=0)

In [25]:
dL_train = MyCustomDataset(L_train, Ly_train)
arc_train_dl1 = DataLoader(dL_train, 32, num_workers=0)

In [26]:
net_arc = ArcNet(EmbeddingNet(), train_classes=len(encoder.keys()))
trainer = pl.Trainer(gpus=0, max_epochs=5)
trainer.fit(net_arc, arc_train_dl1)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name       | Type         | Params
--------------------------------------------
0 | emb_net    | EmbeddingNet | 10.1 M
1 | classifier | ArcLoss      | 241 K 
--------------------------------------------
10.4 M    Trainable params
0         Non-trainable params
10.4 M    Total params
41.423    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 70/70 [00:18<00:00,  3.78it/s, loss=2.47, v_num=50, train_loss=2.100, train_acc=0.969] 


1

In [27]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #KNN 1 5ep

Accuracy = 96.57039711191335%


0.9657039711191335

In [31]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #KNN 1 10ep

Accuracy = 96.20938628158845%


0.9620938628158845

In [74]:
net_arc.eval()
net_arc.emb_net.to('cpu')
knn = train_knn(net_arc.emb_net, is_normalize=True)
test_knn(net_arc.emb_net, knn, is_normalize=True) #Extra

Accuracy = 93.32129963898917%


0.9332129963898917

In [28]:
torch.save(net_arc.state_dict(), "arc_net.pth")

In [29]:
test_net = ArcNet(EmbeddingNet(), train_classes=len(encoder.keys()))
test_net.load_state_dict(torch.load('arc_net.pth'))
test_net.eval()

test_net.emb_net.to('cpu')
knn = train_knn(test_net.emb_net, is_normalize=True)
test_knn(test_net.emb_net, knn, is_normalize=True)

Accuracy = 96.57039711191335%


0.9657039711191335